In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [1]:
!nvidia-smi

Fri Apr 28 22:20:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    48W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [22]:
!pip install transformers
!pip install pytorch_lightning==2.0.2
!pip install sentencepiece
!pip install optuna==3.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.4 MB/s eta 0:00:00


In [9]:
import pandas as pd
import numpy as np
import torch
import pytorch_lightning as pl
import glob
import os
import re
import argparse
from sklearn import metrics
import locale
import gc

import optuna
from optuna.integration import PyTorchLightningPruningCallback
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, AutoTokenizer, AdamW, get_linear_schedule_with_warmup, DistilBertForSequenceClassification

#torch.set_float32_matmul_precision("medium")

In [4]:
!unzip -qq steam-ds.zip -d steam-ds

In [10]:
train_pos_files = glob.glob('steam-ds/train/pos/*.txt')
train_neg_files = glob.glob('steam-ds/train/pos/*.txt')
print(len(train_pos_files), len(train_neg_files))

24000 24000


#Dataset
* Based on T5 Text classification code (https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/t5#resources) 

In [11]:
class SteamDataset(Dataset):
  def __init__(self, tokenizer, data_dir, type_path,  max_len=512):
    self.pos_file_path = os.path.join(data_dir, type_path, 'pos')
    self.neg_file_path = os.path.join(data_dir, type_path, 'neg')
    
    self.pos_files = glob.glob("%s/*.txt" % self.pos_file_path)
    self.neg_files = glob.glob("%s/*.txt" % self.neg_file_path)
    
    self.max_len = max_len
    self.tokenizer = tokenizer

    self.inputs = []
    self.targets = []

    self._build()
  
  def __len__(self):
    return len(self.inputs)

In [12]:
class SteamT5Dataset(SteamDataset):
  def __getitem__(self, index):
    source_ids = self.inputs[index]["input_ids"].squeeze()
    target_ids = self.targets[index]["input_ids"].squeeze()

    src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
    target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

    return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}

  def _build(self):
    self._build_examples_from_files(self.pos_files, 'positive')
    self._build_examples_from_files(self.neg_files, 'negative')
  
  def _build_examples_from_files(self, files, sentiment):
    REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")

    for path in files:
      with open(path, 'r') as f:
        text = f.read()
      
      line = text.strip()
      line = REPLACE_NO_SPACE.sub("", line) 
      line = line

      target = sentiment

       # tokenize inputs
      tokenized_inputs = self.tokenizer([line], max_length=self.max_len, padding='max_length', truncation=True, return_tensors="pt")
       # tokenize targets
      tokenized_targets = self.tokenizer([target], max_length=2, padding='max_length', return_tensors="pt")

      assert tokenized_inputs['input_ids'].shape[1] == 512, "actual shape was " + str(tokenized_inputs['input_ids'].shape)

      self.inputs.append(tokenized_inputs)
      self.targets.append(tokenized_targets)


In [13]:
def get_dataset(tokenizer, type_path, args):
  return SteamT5Dataset(tokenizer=tokenizer, data_dir = args['data_dir'], type_path = type_path, max_len=args.max_seq_length)

#T5 Transformer
* Also adapted from T5 Text classification code (https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/t5#resources), but heavilly modified to newer versions of Pytorch Lightning and Transformers

In [14]:
class TransformerBase(pl.LightningModule):
  def __init__(self):
    super(TransformerBase, self).__init__()
    self.training_step_outputs = []
    self.validation_step_outputs = []
    self.predictions = []
    self.actual = []
    self.validation_step_acc = []
    self.test_step_acc = []

  def train_dataloader(self):
    train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams)
    dataloader = DataLoader(train_dataset, batch_size=self.hparams['train_batch_size'], drop_last=True, shuffle=True, num_workers=self.hparams['num_workers'])
    return dataloader

  def val_dataloader(self):
    val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams)
    return DataLoader(val_dataset, batch_size=self.hparams['eval_batch_size'], num_workers=self.hparams['num_workers'])

  def test_dataloader(self):
    test_dataset = get_dataset(tokenizer=self.tokenizer, type_path="test", args=self.hparams)
    return DataLoader(test_dataset, batch_size=self.hparams['test_batch_size'], num_workers=self.hparams['num_workers'])

In [15]:
class T5Classification(TransformerBase):
  def __init__(self, hparams):
    super(T5Classification, self).__init__()
    self.hparams.update(hparams)
    
    self.model = T5ForConditionalGeneration.from_pretrained(hparams['model_name_or_path'])
    self.tokenizer = AutoTokenizer.from_pretrained(hparams['tokenizer_name_or_path'])

  def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels,
    )
  
  def training_step(self, batch, batch_idx):
    loss = self._step(batch)
    self.training_step_outputs.append(loss)
    self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)

    return {"loss": loss}

  def on_training_epoch_start(self):
    self.training_step_outputs = []
  
  def on_training_epoch_end(self):
    avg_train_loss = torch.stack(self.training_step_outputs).mean()
    self.log("avg_train_loss", prog_bar=True)
    
    return {"avg_train_loss": avg_train_loss}

  def _step(self, batch):
    labels = batch["target_ids"]
    labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        labels=labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]
    return loss

  def validation_step(self, batch, batch_idx):
    loss = self._step(batch)
    self.validation_step_outputs.append(loss)
    self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True)

    #Generate uses pad_token as start token, so remove it after
    outs = self.model.generate(input_ids = batch['source_ids'], attention_mask=batch['source_mask'], max_length=3)
    dec = [self.tokenizer.decode(ids[1:]) for ids in outs]
    target = [self.tokenizer.decode(ids) for ids in batch['target_ids']]
    accuracy = torch.tensor(metrics.accuracy_score(target, dec))

    self.predictions.extend(dec)
    self.actual.extend(target)

    self.log("val_acc", accuracy, on_step=True, on_epoch=True, prog_bar=True)
    self.validation_step_acc.append(accuracy)

    return {"val_loss": loss, "val_acc": accuracy}

  def on_validation_epoch_start(self):
    self.validation_step_outputs = []
    self.validation_step_acc = []
    self.predictions = []
    self.actual = []

  def on_validation_epoch_end(self):
    avg_loss = torch.stack(self.validation_step_outputs).mean()
    avg_acc = torch.stack(self.validation_step_acc).mean()

    f1 = metrics.f1_score(self.actual, self.predictions)

    self.log("val_f1", f1, prog_bar=True)
    self.log("avg_val_loss", avg_loss, prog_bar=True)
    self.log("avg_val_acc", avg_acc, prog_bar=True)

    return {"avg_val_loss": avg_loss, "avg_val_acc": avg_acc, "val_f1": f1}

  def test_step(self, batch, batch_idx):
    #Generate uses pad_token as start token, so remove it after
    outs = self.model.generate(input_ids = batch['source_ids'], attention_mask=batch['source_mask'], max_length=3)
    dec = [self.tokenizer.decode(ids[1:]) for ids in outs]
    target = [self.tokenizer.decode(ids) for ids in batch['target_ids']]
    accuracy = torch.tensor(metrics.accuracy_score(target, dec))

    self.predictions.extend(dec)
    self.actual.extend(target)

    self.log("test_acc", accuracy, on_step=True, prog_bar=True)
    self.test_step_acc.append(accuracy)

    return {"test_acc": accuracy}

  def on_test_epoch_start(self):
    self.test_step_outputs = []
    self.test_step_acc = []
    self.predictions = []
    self.actual = []

  def on_test_epoch_end(self):
    avg_acc = torch.stack(self.test_step_acc).mean()
    f1 = metrics.f1_score(self.actual, self.predictions)
    self.log("test_f1", f1, prog_bar=True)
    self.log("avg_test_acc", avg_acc, prog_bar=True)
    return {"avg_test_acc": avg_acc, "test_f1": f1}

  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams['weight_decay'],
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=self.hparams['learning_rate'], eps=self.hparams['adam_epsilon'])
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=self.hparams['warmup_steps'], num_training_steps=self.trainer.estimated_stepping_batches
    )
    return [optimizer], scheduler
  
  def lr_scheduler_step(self, scheduler, metric):
    scheduler.step()

In [ ]:
args_dict = dict(
    data_dir="steam-ds", # path for data files
    output_dir="t5-steam-ds-sentiment", # path to save the checkpoints
    model_name_or_path='t5-small',
    tokenizer_name_or_path='t5-small',
    max_seq_length=512,
    learning_rate=0.001,
    weight_decay=3e-7,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=8,
    eval_batch_size=8,
    test_batch_size=8,
    num_train_epochs=4,
    gradient_accumulation_steps=16,
    n_gpu=1,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    num_workers=6,
    seed=42,
)

In [ ]:
model = T5Classification(args_dict)

In [ ]:
!mkdir -p t5-steam-ds-sentiment

In [ ]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=args_dict['output_dir'], monitor="avg_val_acc", mode="max", save_top_k=5
)

train_params = dict(
    accumulate_grad_batches=args_dict['gradient_accumulation_steps'],
    max_epochs = args_dict['num_train_epochs'],
    precision= 16 if args_dict['fp_16'] else 32,
    gradient_clip_val=args_dict['max_grad_norm'],
    enable_checkpointing=True,
    callbacks=[checkpoint_callback]
)
trainer = pl.Trainer(**train_params)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loading `train_dataloader` to estimate number of stepping batches.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=4` reached.


In [ ]:
trainer.test(model)

INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       avg_test_acc        │    0.8538333333333333     │
│      test_acc_epoch       │    0.8538333333333333     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc_epoch': 0.8538333333333333, 'avg_test_acc': 0.8538333333333333}]

#T5 Hyperparameter Tuning

In [ ]:
def objective(trial):

  lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
  weight_decay = trial.suggest_float("weight_decay", 1e-8, 1e-2, log=True)
  warmup_steps = trial.suggest_int("warmup_steps", 0, 100, step=10)
  batch_size = trial.suggest_int("batch_size", 4, 16, step=2)
  train_epochs = trial.suggest_int("train_epochs", 1, 4)
  grad_accumulation_steps = trial.suggest_int("gradient_accumulation_steps", 10, 30, step=2)

  args_dict = dict(
    data_dir="steam-ds", # path for data files
    output_dir="t5-steam-ds-sentiment", # path to save the checkpoints
    model_name_or_path='t5-small',
    tokenizer_name_or_path='t5-small',
    max_seq_length=512,
    learning_rate=lr,
    weight_decay=weight_decay,
    adam_epsilon=1e-8,
    warmup_steps=warmup_steps,
    train_batch_size=batch_size,
    eval_batch_size=batch_size,
    test_batch_size=batch_size,
    num_train_epochs=train_epochs,
    gradient_accumulation_steps=grad_accumulation_steps,
    n_gpu=1,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    num_workers=4,
    seed=42,
  )

  checkpoint_callback = pl.callbacks.ModelCheckpoint(
    os.path.join(args_dict["output_dir"],"trial_{}".format(trial.number)), monitor="avg_val_acc", mode="max")
  
  earlystop_callback = pl.callbacks.EarlyStopping(
      monitor="avg_val_acc", mode="max", check_on_train_epoch_end=False, divergence_threshold=0.6, patience=1
    )

  train_params = dict(
    accumulate_grad_batches=args_dict['gradient_accumulation_steps'],
    max_epochs = args_dict['num_train_epochs'],
    precision= 16 if args_dict['fp_16'] else 32,
    gradient_clip_val=args_dict['max_grad_norm'],
    enable_checkpointing=True,
    callbacks=[checkpoint_callback, earlystop_callback],
    limit_train_batches=0.25
  )

  model = T5Classification(args_dict)
  trainer = pl.Trainer(**train_params)

  trainer.fit(model)

  return trainer.callback_metrics["avg_val_acc"]

study = optuna.create_study(direction="maximize", pruner=None)
study.enqueue_trial(
    {"lr": 3e-4, 
     "weight_decay": 1e-8, 
     "warmup_steps": 0, 
     "batch_size": 8, 
     "train_epochs": 2, 
     "gradient_accumulation_steps": 16}
    )

study.optimize(objective, n_trials=25, gc_after_trial=True)

print("Best trial:")
trial = study.best_trial

print("Params:")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

[I 2023-04-28 05:53:58,339] A new study created in memory with name: no-name-80289401-7289-4b5a-8235-ace58acd2219


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loading `train_dataloader` to estimate number of stepping batches.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type              

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

[I 2023-04-28 08:03:02,817] Trial 24 finished with value: 0.0 and parameters: {'lr': 0.0008179805550911611, 'weight_decay': 5.654790795188224e-05, 'warmup_steps': 30, 'batch_size': 14, 'train_epochs': 3, 'gradient_accumulation_steps': 16}. Best is trial 13 with value: 0.8525.


{'train_loss': tensor(9.4500), 'train_loss_step': tensor(10.0912), 'val_loss': tensor(9.5325), 'val_loss_epoch': tensor(9.5325), 'val_acc': tensor(0., dtype=torch.float64), 'val_acc_epoch': tensor(0., dtype=torch.float64), 'avg_val_loss': tensor(9.5353), 'avg_val_acc': tensor(0., dtype=torch.float64), 'train_loss_epoch': tensor(9.4500)}
Best trial:
Params:
    lr: 0.0010260066116536213
    weight_decay: 3.350496690379203e-07
    warmup_steps: 0
    batch_size: 8
    train_epochs: 4
    gradient_accumulation_steps: 10


#DistilBERT

In [16]:
class SteamBERTDataset(SteamDataset):
  def __getitem__(self, index):
    source_ids = self.inputs[index]["input_ids"].squeeze()
    src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze

    target = self.targets[index]

    return {"source_ids": source_ids, "source_mask": src_mask, "labels": target}

  def _build(self):
    self._build_examples_from_files(self.pos_files, 'positive')
    self._build_examples_from_files(self.neg_files, 'negative')

    self.targets = torch.stack(self.targets).squeeze()
  
  def _build_examples_from_files(self, files, sentiment):
    REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")

    for path in files:
      with open(path, 'r') as f:
        text = f.read()
      
      line = text.strip()
      line = REPLACE_NO_SPACE.sub("", line) 
      line = line

      # tokenize inputs
      tokenized_inputs = self.tokenizer([line], max_length=self.max_len, padding='max_length', truncation=True, return_tensors="pt")
      assert tokenized_inputs['input_ids'].shape[1] == 512, "actual shape was " + str(tokenized_inputs['input_ids'].shape)

      if (sentiment == 'positive'):
        target = torch.ones(1)
      else:
        target = torch.zeros(1)

      self.inputs.append(tokenized_inputs)
      self.targets.append(target)


In [17]:
def get_dataset(tokenizer, type_path, args):
  return SteamBERTDataset(tokenizer=tokenizer, data_dir = args['data_dir'], type_path = type_path, max_len=args["max_seq_length"])

In [24]:
class DistilBertClassification(TransformerBase):
  def __init__(self, hparams):
    super(DistilBertClassification, self).__init__()
    self.hparams.update(hparams)
    
    self.model = DistilBertForSequenceClassification.from_pretrained(hparams['model_name_or_path'], num_labels=2)
    self.tokenizer = AutoTokenizer.from_pretrained(hparams['tokenizer_name_or_path'])

  def forward(
      self, input_ids, attention_mask=None, labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        labels=labels,
    )

  def _step(self, batch):
    one_hot = torch.nn.functional.one_hot(batch["labels"].long(), num_classes=2).float()
    one_hot = one_hot.to(self.device)
    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        labels=one_hot
    )

    loss = outputs[0]
    logits = outputs[1]
    return loss, logits

  def training_step(self, batch, batch_idx):
    loss, logits = self._step(batch)
    self.training_step_outputs.append(loss)
    self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)

    return {"loss": loss}

  def on_training_epoch_start(self):
    self.training_step_outputs = []
  
  def on_training_epoch_end(self):
    avg_train_loss = torch.stack(self.training_step_outputs).mean()
    self.log("avg_train_loss", prog_bar=True)
    return {"avg_train_loss": avg_train_loss}

  def validation_step(self, batch, batch_idx):
    loss, logits = self._step(batch)
    self.validation_step_outputs.append(loss)
    self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True)

    pred = torch.argmax(logits, -1).to('cpu')
    target = batch['labels'].to('cpu')
    accuracy = torch.tensor(metrics.accuracy_score(pred, target))

    self.predictions.extend(pred)
    self.actual.extend(target)

    self.log("val_acc", accuracy, on_step=True, on_epoch=True, prog_bar=True)
    self.validation_step_acc.append(accuracy)

    return {"val_loss": loss, "val_acc": accuracy}

  def on_validation_epoch_start(self):
    self.validation_step_outputs = []
    self.validation_step_acc = []
    self.predictions = []
    self.actual = []

  def on_validation_epoch_end(self):
    avg_loss = torch.stack(self.validation_step_outputs).mean()
    avg_acc = torch.stack(self.validation_step_acc).mean()

    f1 = metrics.f1_score(self.actual, self.predictions)
    self.log("val_f1", f1, prog_bar=True)

    self.log("avg_val_loss", avg_loss, prog_bar=True)
    self.log("avg_val_acc", avg_acc, prog_bar=True)

    return {"avg_val_loss": avg_loss, "avg_val_acc": avg_acc, "val_f1": f1}

  def test_step(self, batch, batch_idx):
    loss, logits = self._step(batch)
    pred = torch.argmax(logits, -1).to('cpu')
    target = batch['labels'].to('cpu')
    accuracy = torch.tensor(metrics.accuracy_score(pred, target))

    self.predictions.extend(pred)
    self.actual.extend(target)

    self.log("test_acc", accuracy, on_step=True, prog_bar=True)
    self.test_step_acc.append(accuracy)

    return {"test_acc": accuracy}

  def on_test_epoch_start(self):
    self.validation_step_outputs = []
    self.validation_step_acc = []
    self.predictions = []
    self.actual = []

  def on_test_epoch_end(self):
    avg_acc = torch.stack(self.test_step_acc).mean()
    f1 = metrics.f1_score(self.actual, self.predictions)
    self.log("test_f1", f1, prog_bar=True)
    self.log("avg_test_acc", avg_acc, prog_bar=True)
    return {"avg_test_acc": avg_acc, "test_f1": f1}

  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams['weight_decay'],
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=self.hparams['learning_rate'], eps=self.hparams['adam_epsilon'])
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=self.hparams['warmup_steps'], num_training_steps=self.trainer.estimated_stepping_batches
    )
    return [optimizer], scheduler
  
  def lr_scheduler_step(self, scheduler, metric):
    scheduler.step()

In [27]:
args_dict = dict(
    data_dir="steam-ds", # path for data files
    output_dir="distilbert-steam-ds-sentiment", # path to save the checkpoints
    model_name_or_path="distilbert-base-uncased",
    tokenizer_name_or_path="distilbert-base-uncased",
    max_seq_length=512,
    learning_rate=1e-4,
    weight_decay=2.4e-8,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=6,
    eval_batch_size=6,
    test_batch_size=6,
    num_train_epochs=2,
    gradient_accumulation_steps=14,
    n_gpu=1,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    num_workers=4,
    seed=42,
  )

In [28]:
model = DistilBertClassification(args_dict)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

In [16]:
!mkdir -p distilbert-steam-ds-sentiment

In [29]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=args_dict['output_dir'], monitor="avg_val_acc", mode="max", save_top_k=5
)

train_params = dict(
    accumulate_grad_batches=args_dict['gradient_accumulation_steps'],
    max_epochs = args_dict['num_train_epochs'],
    precision= 16 if args_dict['fp_16'] else 32,
    gradient_clip_val=args_dict['max_grad_norm'],
    enable_checkpointing=True,
    callbacks=[checkpoint_callback],
)
trainer = pl.Trainer(**train_params)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [30]:
trainer.fit(model)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /content/distilbert-steam-ds-sentiment exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loading `train_dataloader` to estimate number of stepping batches.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                                | Params
--------------------------------------------------------------
0 | model | DistilBertForSequenceClassification | 67.0 M
--------------------------------------------------------------
67.0 M    Trainable params
0         Non-trainable params
67.0 M    Total params
267.820   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=2` reached.


In [31]:
trainer.test(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       avg_test_acc        │    0.8596666666666667     │
│      test_acc_epoch       │    0.8596666666666667     │
│          test_f1          │    0.8588669125041904     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc_epoch': 0.8596666666666667,
  'test_f1': 0.8588669125041904,
  'avg_test_acc': 0.8596666666666667}]

In [20]:
def objective(trial):
  lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
  weight_decay = trial.suggest_float("weight_decay", 1e-8, 1e-2, log=True)
  warmup_steps = trial.suggest_int("warmup_steps", 0, 100, step=10)
  batch_size = trial.suggest_int("batch_size", 4, 16, step=2)
  train_epochs = trial.suggest_int("train_epochs", 1, 4)
  grad_accumulation_steps = trial.suggest_int("gradient_accumulation_steps", 10, 30, step=2)

  args_dict = dict(
    data_dir="steam-ds", # path for data files
    output_dir="distilbert-steam-ds-sentiment", # path to save the checkpoints
    model_name_or_path="distilbert-base-uncased",
    tokenizer_name_or_path="distilbert-base-uncased",
    max_seq_length=512,
    learning_rate=lr,
    weight_decay=weight_decay,
    adam_epsilon=1e-8,
    warmup_steps=warmup_steps,
    train_batch_size=batch_size,
    eval_batch_size=batch_size,
    test_batch_size=batch_size,
    num_train_epochs=train_epochs,
    gradient_accumulation_steps=grad_accumulation_steps,
    n_gpu=1,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    num_workers=4,
    seed=42,
  )

  checkpoint_callback = pl.callbacks.ModelCheckpoint(
    os.path.join(args_dict["output_dir"],"trial_{}".format(trial.number)), monitor="avg_val_acc", mode="max")
  
  earlystop_callback = pl.callbacks.EarlyStopping(
      monitor="avg_val_acc", mode="max", check_on_train_epoch_end=False, divergence_threshold=0.6, patience=1
    )
  
  train_params = dict(
    accumulate_grad_batches=args_dict['gradient_accumulation_steps'],
    max_epochs = args_dict['num_train_epochs'],
    precision= 16 if args_dict['fp_16'] else 32,
    gradient_clip_val=args_dict['max_grad_norm'],
    enable_checkpointing=True,
    callbacks=[checkpoint_callback, earlystop_callback],
  )

  model = DistilBertClassification(args_dict)
  trainer = pl.Trainer(**train_params)

  trainer.fit(model)
  return trainer.callback_metrics["avg_val_acc"]

torch.set_float32_matmul_precision('medium')
study = optuna.create_study(direction="maximize", pruner=None)
study.enqueue_trial(
    {"lr": 3e-4, 
     "weight_decay": 1e-8, 
     "warmup_steps": 0, 
     "batch_size": 8, 
     "train_epochs": 2, 
     "gradient_accumulation_steps": 16}
    )

study.optimize(objective, n_trials=25, gc_after_trial=True)

print("Best trial:")
trial = study.best_trial

print("Params:")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

[I 2023-04-28 22:54:43,831] A new study created in memory with name: no-name-c845d29d-733f-4550-b7f8-7c08f6d03cdf
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpo

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

[I 2023-04-29 01:13:12,646] Trial 24 finished with value: 0.477 and parameters: {'lr': 0.00015034407643404617, 'weight_decay': 1.0927143784594459e-07, 'warmup_steps': 10, 'batch_size': 4, 'train_epochs': 3, 'gradient_accumulation_steps': 18}. Best is trial 23 with value: 0.8616666666666667.


Best trial:
Params:
    lr: 0.00015244618212887392
    weight_decay: 2.4120724200537892e-08
    warmup_steps: 0
    batch_size: 6
    train_epochs: 2
    gradient_accumulation_steps: 14
